In [1]:
!pip install pymystem3

In [2]:
import numpy as np 
import pandas as pd 

from sklearn.metrics import roc_auc_score
from pymystem3 import Mystem
import pickle

from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [3]:
seed=1234
tf.random.set_seed(seed)

In [4]:
df = pd.read_csv('/kaggle/input/testavito/train.csv')

In [5]:
%%time

m = Mystem()
def lemmatize(text):
    return m.lemmatize(text)
#    return "".join(m.lemmatize(text))


df['description'] = df['description'].apply(lemmatize)

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


CPU times: user 11min 53s, sys: 2min 57s, total: 14min 50s
Wall time: 52min 26s


In [6]:
#Разделение выборки
train, test = train_test_split(df, test_size = 0.1, random_state = 1234)

print("Train 80%", train.shape)
print('Test 10%', test.shape)

Train 80% (810000, 9)
Test 20% (90000, 9)


In [7]:
train_target = train['is_bad']
test_target = test['is_bad']

In [10]:
#отчищаем память
df = None

In [8]:
%%time
max_words = 8000
max_len = 350
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(train['description'])
#трейн матрица
sequences = tok.texts_to_sequences(train['description'])
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)


CPU times: user 3min 44s, sys: 0 ns, total: 3min 44s
Wall time: 3min 47s


In [11]:
#тест матрица
test_sequences = tok.texts_to_sequences(test['description'])
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [12]:
#отчищаем память
train = None
#тест отсавляем, там категории для финальной метрики еще

In [14]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [15]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 250)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 250, 50)           250000    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                

In [16]:
model.fit(sequences_matrix,train_target,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=2)])

Epoch 1/10
5063/5063 [==============================] - 104s 20ms/step - loss: 0.2304 - accuracy: 0.9219 - val_loss: 0.1922 - val_accuracy: 0.9383
Epoch 2/10
5063/5063 [==============================] - 103s 20ms/step - loss: 0.1856 - accuracy: 0.9404 - val_loss: 0.1813 - val_accuracy: 0.9428
Epoch 3/10
5063/5063 [==============================] - 102s 20ms/step - loss: 0.1723 - accuracy: 0.9443 - val_loss: 0.1699 - val_accuracy: 0.9451
Epoch 4/10
5063/5063 [==============================] - 104s 21ms/step - loss: 0.1658 - accuracy: 0.9461 - val_loss: 0.1667 - val_accuracy: 0.9457
Epoch 5/10
5063/5063 [==============================] - 103s 20ms/step - loss: 0.1608 - accuracy: 0.9475 - val_loss: 0.1657 - val_accuracy: 0.9464
Epoch 6/10
5063/5063 [==============================] - 102s 20ms/step - loss: 0.1576 - accuracy: 0.9485 - val_loss: 0.1741 - val_accuracy: 0.9462
Epoch 7/10
5063/5063 [==============================] - 105s 21ms/step - loss: 0.1552 - accuracy: 0.9493 - val_loss: 0

In [17]:
model.save("model.h5")

In [18]:

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
#with open('tokenizer.pickle', 'rb') as handle:
#    tok = pickle.load(handle)

In [19]:
pred = model.predict(test_sequences_matrix)

In [21]:
roc_auc_score(test_target, pred)

0.9710264809988374

In [20]:
test['pred'] = pred
metric_list = []
for i in test['category'].unique():
    predictions = test[test['category']==i]['pred'].values
    real = test[test['category']==i]['is_bad'].values
    metric_list.append(roc_auc_score(real, predictions ))
   
    
    
print( pd.Series(metric_list).mean())

TypeError: 'NoneType' object does not support item assignment